In [9284]:
import pandas as pd
import datetime
import json

In [9285]:
def get_compressedData(listContent_One : list, listContent_Two : list, listHead : list):
    listVar = []
    for i in range(min(len(listContent_One), len(listContent_Two))):
        listVar.append( {listHead[0]:listContent_One[i], listHead[1]:listContent_Two[i]} )
    return listVar

In [9286]:
def save_fileInJSON(content ,strRoot : str, mode = 'w'):
    with open(strRoot,mode) as file:
        file.write(json.dumps(content))

In [9287]:
def get_SystSampl( TotalCount, flRate = 0.45):
    if type(TotalCount) == int:
         return round(TotalCount*flRate)
    else:
        intVar = TotalCount.shape[0]
        return round(intVar*flRate)

In [9288]:
data = pd.read_csv('RTG_CSV.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)

In [9289]:
data.loc[data.gender == 'М', "gender"] = 'M'
data.loc[data.gender == 'Ж', "gender"] = 'F'

In [9290]:
data

,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,fe60b594364f9f636266ed1ef4f89c32,F,1985,2020-01-29,R,Гусь-Хрустальный,0,Договор на текущий счет для дебетовой карты,a83cbc33f164deae41b3339141923cc7,Visa Platinum Rewards,2019,2021,16600,71,39700,25700,220600,201000,dc
1,3012cabca5885ed53d348d6e57dab5de,M,1951,2009-09-24,R,Ишимбай,0,Договор на текущий счет для дебетовой карты,8dae17a830bb05d151034df618efbcad,VISA Classic,2019,-1,4900,2,38000,26800,56300,31300,dc
2,d871ef96820b6c7a1ada8e01a772724e,F,1982,2006-12-25,R,Юрга,0,Договор на текущий счет для дебетовой карты,429ca8c43cdc5f3d8c9efc9069ee3fa6,NaN,-1,-1,0,0,0,0,0,0,dc
3,0bacbc2abd314f9fac031d249e485a7b,M,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7f4adb9187b867ebea15503298437a3,Visa Rewards,2021,-1,47800,12,0,0,51500,51500,cc
4,0bacbc2abd314f9fac031d249e485a7b,M,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7d13882154e965dca7070eb8d626f77,Eurocard/MasterCard World,2018,2022,47800,12,0,0,51500,51500,cc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,9acb18399b830419bac9653b52754646,M,1986,2015-05-20,R,Уфа,0,Договор на текущий счет для дебетовой карты,d090cb20c22e2643ec366daf8d26d646,NaN,-1,-1,0,0,0,0,0,0,dc
16309,8b3206ca8d77aa2446824971f47c531b,F,1996,2022-04-28,R,Казань,30000,Кредитная карта,5a0041efbd5832765caf8b9f861bffa3,NaN,-1,-1,22700,7,0,0,22700,22700,dc
16310,d69d3201104ab393c0638839a7284ad7,M,1953,2018-08-08,R,Пятигорск,200000,Кредитная карта,a78c73e84e893179de17071583cf5084,NaN,-1,-1,0,0,0,300,9900,10100,dc
16311,a936fb30e2268d2874cdcaa221032915,M,1997,2022-05-12,R,Кемерово,15000,Кредитная карта,e3304eaf55891a175b6abf8030b99da1,NaN,-1,-1,15000,33,0,0,15000,15000,dc


Суммарный остаток на карте

In [9291]:
avrBalance = data
avrBalance = avrBalance.query("current_balance_avg_sum > 0")
avrBalance = avrBalance.current_balance_avg_sum
avrBalance = avrBalance.sample(n=get_SystSampl(avrBalance))


In [9292]:
avrBalance = avrBalance.median()
avrBalance = avrBalance.tolist()

In [9294]:
save_fileInJSON([{'avrBalance': avrBalance}], 'avrBalance.json')

Мужчины / Женщины

In [9295]:
ManToWoman = data
ManToWoman = ManToWoman.drop_duplicates(subset=['client_id'])
ManToWoman = ManToWoman.sample(n=get_SystSampl(ManToWoman))
ManToWoman = ManToWoman['gender'].value_counts()

In [9296]:
listGender   = [(ManToWoman.index)[0],(ManToWoman.index)[1]]
listPercent  = [[ManToWoman.values][0][0].tolist(), [ManToWoman.values][0][1].tolist()]
listHead = ['Gender','Percent']
save_fileInJSON(  get_compressedData( listGender, listPercent, listHead), 'Gender.json')

Количество приобретённых карт за год

In [9297]:
now = datetime.datetime.now()

In [9298]:
cardTypeDF = data.query(f"start_date >= {now.year}-1")
cardTypeDF = cardTypeDF.sample(n=get_SystSampl(cardTypeDF))

In [9299]:
listCardType = cardTypeDF['card_type_name'].unique()

In [9300]:
cardTypeDF = cardTypeDF['card_type_name'].value_counts()

In [9301]:
sum = cardTypeDF.values.sum()

In [9302]:
listCreditSys = ['МИР', 'MIR', 'Visa', 'MasterCard','Maestro','Other']
listCreditSysCount = [0]*len(listCreditSys)

In [9303]:
for i in range(len(cardTypeDF.index)):
    for j in range(len(listCreditSys)):
        if (listCreditSys[j] in cardTypeDF.index[i]) or (listCreditSys[j] in cardTypeDF.index[i].capitalize()) : listCreditSysCount[j] += cardTypeDF.values[i]

In [9304]:
var = 0
for i in range(len(listCreditSysCount)): var += listCreditSysCount[i]

In [9305]:
listCreditSysCount[len(listCreditSysCount[:-1])] = sum - var

In [9306]:
listCreditSys = listCreditSys[1:]
listCreditSysCount[1] = listCreditSysCount[0] + listCreditSysCount[1]
listCreditSysCount = listCreditSysCount[1:]

In [9307]:
for i in range(len(listCreditSysCount)): listCreditSysCount[i] = round(listCreditSysCount[i]*100/sum, 1)

In [9308]:
save_fileInJSON(get_compressedData(listCreditSys, listCreditSysCount,['SystemName','Percent']), 'NewBankCard.json')

Кол-во действующих банковских карт у человека в среднем

In [9309]:
ValueCardData = data
ValueCardData = ValueCardData.query("start_date != -1 & fact_close_date == -1")
ValueCardData = ValueCardData.drop(['purchase_sum','create_date', "create_date",'purchase_count','current_balance_avg_sum','current_balance_sum','current_debit_turn_sum','current_credit_turn_sum','contract_sum','gender','birth_date'], axis=1)
ValueCardData = ValueCardData.drop(['card_type_name', 'start_date', 'fact_close_date', 'card_type', 'product_category_name', 'city', 'nonresident_flag'], axis=1)

In [9310]:
ValueCardData = ValueCardData.value_counts('client_id')

In [9311]:
ValueCardData = ValueCardData.drop_duplicates()
ValueCardData = ValueCardData.sample(n=get_SystSampl(ValueCardData))

In [9312]:
listCreditCard = [1,2,3,"4+"]
listCreditCardCount = [0]*len(listCreditCard)

In [9313]:
for i in range(len(ValueCardData.index)):
        if ValueCardData[i] == 1:
            listCreditCardCount[0] +=1
        elif ValueCardData[i] == 2:
            listCreditCardCount[1] +=1
        elif ValueCardData[i] == 3:
            listCreditCardCount[2] +=1
        else:
            listCreditCardCount[3] +=1

In [9314]:
intSum = 0
for i in range(len(listCreditCardCount)) : intSum += listCreditCardCount[i]

In [9315]:
for i in range(len(listCreditCardCount)):
    listCreditCardCount[i] = round(listCreditCardCount[i] * 100 / intSum,2)

In [9316]:
save_fileInJSON(get_compressedData(listCreditCard, listCreditCardCount,['CardType','CardCount']), 'CreditCardCount.json')

Возраст пользователей 

In [9317]:
ValueAgeData = data.query("start_date != -1 & fact_close_date == -1")
ValueAgeData = ValueAgeData.drop(['purchase_sum','create_date', "create_date",'purchase_count','current_balance_avg_sum','current_balance_sum','current_debit_turn_sum','current_credit_turn_sum','contract_sum','gender'], axis=1)
ValueAgeData = ValueAgeData.drop(['card_type_name', 'start_date', 'fact_close_date', 'card_type', 'product_category_name', 'city', 'nonresident_flag','card_id'], axis=1)
ValueAgeData = ValueAgeData.sample(n=get_SystSampl(ValueAgeData))

In [9318]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [9319]:
intMaxAge = 66

In [9320]:
listAgeZone = getlistEqualPartsOfNumberParts(100,14)

In [9321]:
listCountAgeZone = [0] * len(listAgeZone)
listSrtAgeZone = []
if listAgeZone[0] < 14: intLeft = 14-listAgeZone[0]
intRigth = listAgeZone[0]
intSum = 0

In [9322]:
for i in range(len(listAgeZone)-1):
    intLeft += listAgeZone[i]
    intRigth += listAgeZone[i+1]
    listCountAgeZone[i] += ValueAgeData.query(f"{now.year-intLeft} >= birth_date > {now.year-intRigth}").shape[0]
    intSum += listCountAgeZone[i]
    listSrtAgeZone.append( str(str(intLeft) + '-' + str(intRigth)) )
    if i == 0 : intLeft -= 14-listAgeZone[0]


In [9323]:
for i in range(len(listCountAgeZone)): listCountAgeZone[i] = round(listCountAgeZone[i]   * 100 / intSum,2)

In [9324]:
save_fileInJSON(get_compressedData(listSrtAgeZone[:-1], listCountAgeZone, ['Age','Percent'] ), 'Age.json')

Сколько средний остаток за месяц по кредитным картам

In [9325]:
turnBalanceData = data
turnBalanceData = turnBalanceData.drop([ 'gender', 'birth_date', 'create_date', 'nonresident_flag', 'city', 'contract_sum', 'card_type_name', 'start_date', 'fact_close_date'], axis=1)
balanceData = turnBalanceData.drop_duplicates(subset='card_id')
creditTurnBalanceData = turnBalanceData.query("current_credit_turn_sum > 0 & product_category_name == 'Договор на текущий счет для дебетовой карты'")
creditTurnBalanceData = creditTurnBalanceData.sample(n=get_SystSampl(creditTurnBalanceData))
debetTurnBalanceData  = turnBalanceData.query("current_debit_turn_sum > 0  & product_category_name == 'Кредитная карта'")
debetTurnBalanceData = debetTurnBalanceData.sample(n=get_SystSampl(debetTurnBalanceData))
#ValueAgeData = ValueAgeData.sample(n=get_SystSampl(ValueAgeData))

In [9326]:
[debetTurnBalanceData.shape[0], creditTurnBalanceData.shape[0]]

[699, 4905]

In [9327]:
medianDebetTurnBalanceData = debetTurnBalanceData['current_debit_turn_sum'].median()
medianCreditTurnBalanceData = creditTurnBalanceData['current_credit_turn_sum'].median()

In [9328]:
save_fileInJSON([ { 'MedianDebetTurnSum':int(medianDebetTurnBalanceData),'MedianCreditTurnSum':int(medianCreditTurnBalanceData) } ], 'MedianBalance.json')

Среднее время пользование банком

In [9329]:
AvrTimeUse = data
AvrTimeUse = AvrTimeUse[ ['client_id', 'card_id','start_date', 'fact_close_date'] ]
AvrTimeUse = AvrTimeUse.query("start_date > 0").sample(n=get_SystSampl(AvrTimeUse))
AvrTimeUse['Sum'] = 0
AvrTimeUse.loc[((AvrTimeUse.fact_close_date == -1)  ), "fact_close_date"] = now.year
AvrTimeUse.Sum = AvrTimeUse.fact_close_date - AvrTimeUse.start_date

In [9330]:
medianAvrTimeUse = AvrTimeUse.Sum.median()

In [9331]:
medianAvrTimeUse

1.0

In [9332]:
save_fileInJSON([{"MidAvrBankUseTime":medianAvrTimeUse}],'MidAvrBankUseTime.json')